<a href="https://colab.research.google.com/github/Arjavjain100/TOS-Summarization/blob/main/Extractive_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing required libraries

In [150]:
!pip install pytextrank
!spacy download en_core_web_sm
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-22 17:34:25.309674: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-22 17:34:25.309881: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-22 17:34:25.309906: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries.

In [1]:
!git clone "https://github.com/Arjavjain100/TOS-Summarization.git"

Cloning into 'TOS-Summarization'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 34 (delta 13), reused 12 (delta 5), pack-reused 0
Unpacking objects: 100% (34/34), 376.39 KiB | 1.54 MiB/s, done.


### Summarization using TextRank

In [151]:
import pandas as pd
import spacy
import pytextrank
import numpy as np
from google.colab import files
from rouge import Rouge

In [3]:
data = pd.read_csv('/content/TOS-Summarization/Dataset/all_v1_transpose.csv')

In [4]:
data.head()

,doc,id,original_text,reference_summary,title,uid,case_code,case_text,note,title_code,title_text,urls,tldr_code,tldr_text
0,Pokemon GO Terms of Service,5786730a6cca83a54c0035b7,welcome to the pokémon go video game services ...,hi.,NaN,legalsum01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pokemon GO Terms of Service,57866df76cca83a54c0035a1,by using our services you are agreeing to thes...,by playing this game you agree to these terms....,Agreement To Terms,legalsum02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pokemon GO Terms of Service,5786730a6cca83a54c0035b6,if you want to use certain features of the ser...,you have to use google pokemon trainer club or...,Eligibility and Account Registration,legalsum03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Pokemon GO Terms of Service,57866df76cca83a54c0035a0,during game play please be aware of your surro...,don t die or hurt others and if you do it s no...,Safe Play,legalsum04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Pokemon GO Terms of Service,57866df76cca83a54c00359f,subject to your compliance with these terms ni...,don t copy modify resell distribute or reverse...,Rights in App,legalsum05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [161]:
X = data['original_text']

In [162]:
# load a spaCy model, depending on language, scale, etc.
summarizer = spacy.load("en_core_web_sm")
# add PyTextRank to the spaCy pipeline
summarizer.add_pipe("textrank")

In [163]:
def summarize(text):
  doc = summarizer(text)
  result = ""
  for sent in doc._.textrank.summary(limit_phrases=15, limit_sentences=2):
    result += str(sent)

  return result

In [164]:
X[42]

'account termination policy youtube will terminate a user s access to the service if under appropriate circumstances the user is determined to be a repeat infringer. youtube reserves the right to decide whether content violates these terms of service for reasons other than copyright infringement such as but not limited to pornography obscenity or excessive length. youtube may at any time without prior notice and in its sole discretion remove such content and or terminate a user s account for submitting such material in violation of these terms of service.'

In [165]:
summarize(X[42])

'youtube reserves the right to decide whether content violates these terms of service for reasons other than copyright infringement such as but not limited to pornography obscenity or excessive length.youtube may at any time without prior notice and in its sole discretion remove such content and or terminate a user s account for submitting such material in violation of these terms of service.'

In [166]:
rouge = Rouge()

In [170]:
scores = rouge.get_scores(data['reference_summary'][42], summarize(X[42]))

In [171]:
scores

[{'rouge-1': {'r': 0.12, 'p': 0.2857142857142857, 'f': 0.16901408034120222},
  'rouge-2': {'r': 0.03333333333333333,
   'p': 0.09090909090909091,
   'f': 0.04878048387864398},
  'rouge-l': {'r': 0.1, 'p': 0.23809523809523808, 'f': 0.1408450662566952}}]

In [172]:
X = X.to_frame()

In [173]:
X['generated_summary'] = np.zeros(X.shape[0])

In [174]:
X['generated_summary'] = X['original_text'].apply(summarize)

In [175]:
X = pd.concat([X,data['reference_summary']],axis=1)

In [177]:
X['rouge'] = np.zeros(X.shape[0])

In [181]:
X['rouge'] = X[['generated_summary','reference_summary']].apply(lambda x: rouge.get_scores(*x), axis=1)

In [182]:
X.head()

,original_text,generated_summary,reference_summary,rouge
0,welcome to the pokémon go video game services ...,to make these pokémon go terms of service the ...,hi.,"[{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, '..."
1,by using our services you are agreeing to thes...,if you are the parent or legal guardian of a c...,by playing this game you agree to these terms....,"[{'rouge-1': {'r': 0.7, 'p': 0.35, 'f': 0.4666..."
2,if you want to use certain features of the ser...,parents of children under the age of 13 unders...,you have to use google pokemon trainer club or...,"[{'rouge-1': {'r': 0.53125, 'p': 0.2125, 'f': ..."
3,during game play please be aware of your surro...,without limiting the foregoing you agree that ...,don t die or hurt others and if you do it s no...,"[{'rouge-1': {'r': 0.3333333333333333, 'p': 0...."
4,subject to your compliance with these terms ni...,if you accessed or downloaded the app from any...,don t copy modify resell distribute or reverse...,"[{'rouge-1': {'r': 0.18181818181818182, 'p': 0..."


In [183]:
X.to_csv("textrank_summaries.csv")

In [184]:
files.download('textrank_summaries.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>